In [2]:
import feather
import numpy as np 
import pandas as pd
from sklearn.preprocessing import (OneHotEncoder,StandardScaler)
from sklearn.pipeline import Pipeline

from sklearn.model_selection import (
    GridSearchCV)

In [3]:
data_wide_train= feather.read_dataframe('walmart_wide_train.feather')
data_original=pd.read_csv("train.csv")

data_wide_test= feather.read_dataframe('walmart_wide_test.feather')
data_original_test=pd.read_csv("test.csv")

In [ ]:
data_wide_train

In [4]:
from scipy import sparse
from scipy.sparse import hstack


In [5]:
def create_sparse_matrix(original_data):
    dummies = pd.get_dummies(original_data.FinelineNumber)
    dummies = dummies[data_original[data_original.FinelineNumber < 200].FinelineNumber.unique()]
    data_visit = original_data[["VisitNumber"]]
    dummy_data = pd.concat([dummies, data_visit], axis=1)
    dummy_data = dummy_data.groupby("VisitNumber")
    total_dummy_data = dummy_data.aggregate(np.sum)

    dummies = pd.get_dummies(original_data.FinelineNumber)
    for x in range(200,9000,100):
        dummies1 = dummies[data_original[(data_original.FinelineNumber >= x) & (data_original.FinelineNumber < x + 100)].FinelineNumber.unique()]
        data_visit = original_data[["VisitNumber"]]
        dummy_data = pd.concat([dummies1, data_visit], axis=1)
        dummy_data = dummy_data.groupby("VisitNumber")
        dummy_data = dummy_data.aggregate(np.sum)
        total_dummy_data = hstack((total_dummy_data, dummy_data))
    return total_dummy_data

In [6]:
#Creo la spare matrix de finite number
spare_matrix_finite_train = create_sparse_matrix(data_original)


In [8]:
spare_matrix_finite_train


<95674x4933 sparse matrix of type '<class 'numpy.uint8'>'
	with 525471 stored elements in COOrdinate format>

In [9]:
#Creo la spare matrix de mis datos wide

data_sparse_train = sparse.bsr_matrix(data_wide_train.astype(np.float))


In [11]:
#Finalmente hago un merge
data_total_train = hstack((data_sparse_train, spare_matrix_finite_train))


In [12]:
data_total_train

<95674x5011 sparse matrix of type '<class 'numpy.float64'>'
	with 1228246 stored elements in COOrdinate format>